# baseline only mri 加重采样

In [22]:
import os 
os.environ['CUDA_VISIBLE_DEVICES'] = "0" 
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
import matplotlib.pyplot as plt
import seaborn as sns
import hiddenlayer as hl
import torch
import torch.nn as nn
from torch.optim import SGD,Adam
import torch.utils.data as Data
from torchvision import models
from  torchvision import transforms
from  torchvision.datasets import ImageFolder
import pickle as pkl
import torchvision.models as models
from torch.utils.data.sampler import WeightedRandomSampler
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [23]:
mp = pkl.load(open("/home/gc/gechang/gec_multi_fusion/end_to_end/train.pkl","rb"),encoding='iso-8859-1')

In [24]:
mci = 0
nc = 1
ad = 2
for index,row in mp.iterrows():
    if(row["Group"] == 0):
        mci += 1
    if(row["Group"] == 1):
        nc += 1
    if(row["Group"] == 2):
        ad += 1
print(mci,nc,ad)

450 237 113


In [25]:
class FineTuneResnet50(nn.Module):
    def __init__(self, num_class=3):
        super(FineTuneResnet50, self).__init__()
        self.num_class = num_class
        resnet50_net_MRI = models.resnet50(pretrained=False)
        self.features_MRI = nn.Sequential(*list(resnet50_net_MRI.children())[:-1])
        self.fc_comb = nn.Sequential(
            nn.Linear(2048,3)
        )

    

    def forward(self, MRI):
        MRI = self.features_MRI(MRI)
        MRI = MRI.view(MRI.size(0),-1)
        output = self.fc_comb(MRI)
        return output

In [26]:
MyResnet = FineTuneResnet50()

In [27]:
#定义优化器
optimizer = torch.optim.Adam(MyResnet.parameters(),lr=0.00001)
loss_func = nn.CrossEntropyLoss()#损失函数

In [28]:
#记录训练过程指标
historyl = hl.History()
#使用Canves进行可视化

canvasl = hl.Canvas()

In [29]:
from torch.utils.data import DataLoader
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, root,transform_pet = None):
        super(MyDataset, self).__init__()
        MRI_PET_match_all = pkl.load(open(root,"rb"),encoding='iso-8859-1')
        MRI = []
        PET = []
        groups = []
        for index,row in MRI_PET_match_all.iterrows():
            MRI.append(row['MRI_img_array'])
            PET.append(row['PET_img_array'])
            group_ = torch.tensor(row['Group'],dtype=torch.float)
            groups.append(group_)
        self.MRI = MRI
        self.PET = PET
        self.groups = groups  
        self.transform_pet = transform_pet

    def __getitem__(self, index):
        mri =torch.from_numpy(self.MRI[index].transpose([2,0,1])).float().to(DEVICE)
        pet = torch.from_numpy(self.PET[index].transpose([2,0,1])).float().to(DEVICE)
        # mri = self.transform_pet(mri)
        # pet = self.transform_pet(pet)
        group = self.groups[index].to(DEVICE)
        return pet,group


    def get_classes_for_all_imgs(self):
        return self.groups
        
    def __len__(self):
        return len(self.MRI)

# not 16
# train_mean_mri = [4.1620684, 4.1620684, 4.1620684]
# train_std_mri = [5.2131376, 5.2131376, 5.2131376]
# train_mean_pet = [4.081158, 4.081158, 4.081158] 
# train_std_pet = [5.1888165, 5.1888165, 5.1888165]

# test_mean_mri = [4.1623616, 4.1623616, 4.1623616]
# test_std_mri = [5.2136188, 5.2136188, 5.2136188]
# test_mean_pet = [4.106387, 4.106387, 4.106387]
# test_std_pet = [5.18535, 5.18535, 5.18535]
    
# only one   
# train_mean_mri = [4.240719, 4.240719, 4.240719]
# train_std_mri = [5.254782, 5.254782, 5.254782]
# train_mean_pet = [4.106597, 4.106597, 4.106597] 
# train_std_pet = [5.21822, 5.21822, 5.21822]

# test_mean_mri = [4.364004, 4.364004, 4.364004]
# test_std_mri = [5.4076633, 5.4076633, 5.4076633] 
# test_mean_pet = [4.2559123, 4.2559123, 4.2559123]
# test_std_pet = [5.373126, 5.373126, 5.373126]

# valid_mean_mri = [4.7232037, 4.7232037, 4.7232037]
# valid_std_mri =  [5.8529644, 5.8529644, 5.8529644] 
# valid_mean_pet = [4.5747848, 4.5747848, 4.5747848]
# valid_std_pet = [5.809622, 5.809622, 5.809622]  

#16-in-1
train_mean_mri = [4.176061, 4.176061, 4.176061]
train_std_mri = [5.231413, 5.231413, 5.231413]
train_mean_pet = [4.017313, 4.017313, 4.017313] 
train_std_pet = [5.1714053, 5.1714053, 5.1714053]

test_mean_mri = [4.3262687, 4.3262687, 4.3262687]
test_std_mri = [5.419836, 5.419836, 5.419836] 
test_mean_pet = [4.1623745, 4.1623745, 4.1623745]
test_std_pet = [5.3586817, 5.3586817, 5.3586817]

valid_mean_mri = [4.7232037, 4.7232037, 4.7232037]
valid_std_mri =  [5.8529644, 5.8529644, 5.8529644] 
valid_mean_pet = [4.5747848, 4.5747848, 4.5747848]
valid_std_pet = [5.809622, 5.809622, 5.809622]
    

train_transform_mri = transforms.Compose([
    transforms.Normalize(train_mean_mri,train_std_mri),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
    
])

train_transform_pet = transforms.Compose([
    transforms.Normalize(train_mean_pet,train_std_pet),
    transforms.RandomVerticalFlip(),
    transforms.RandomHorizontalFlip(),
])

test_transform_mri = transforms.Compose([
    
    transforms.Normalize(test_mean_mri,test_std_mri),
])

test_transform_pet = transforms.Compose([
   
    transforms.Normalize(test_mean_pet,test_std_pet)
])


train_data = MyDataset("/home/gc/gechang/gec_multi_fusion/end_to_end/train.pkl",transform_pet =train_transform_mri)
test_data = MyDataset("/home/gc/gechang/gec_multi_fusion/end_to_end/test.pkl",transform_pet =test_transform_mri)

# 数据集中，每一类的数目。
#class_sample_counts = [5568,3297,2338] #not 16
class_sample_counts = [450,237,113] #only one
weights = 1./ torch.tensor(class_sample_counts, dtype=torch.float)
# 这个 get_classes_for_all_imgs是关键
train_targets = train_data.get_classes_for_all_imgs()
samples_weights = weights[train_targets]

sampler = WeightedRandomSampler(weights=samples_weights, num_samples=len(samples_weights), replacement=True)



train_loader = DataLoader(train_data, batch_size=4, sampler = sampler,shuffle=False)
test_loader = DataLoader(test_data, batch_size=4)


In [30]:
#对模型进行迭代训练，对所有的数据训练epoch轮
for epoch in range(140):
    train_loss_epoch = 0
    val_loss_epoch = 0
    train_corrects = 0
    val_corrects = 0
    #对训练数据的加载器进行迭代计算
    MyResnet.train().cuda()
    for step,(mri,group) in enumerate(train_loader):
        ##计算每个batch的损失
        output = MyResnet(mri)
        loss = loss_func(output,group.long())#交叉熵损失函数
        pre_lab = torch.argmax(output,1).to(DEVICE)
        optimizer.zero_grad()#每个迭代步的梯度初始化为0
        loss.backward()#损失的后向传播，计算梯度
        optimizer.step()#使用梯度进行优化
        train_loss_epoch += loss.item()*group.size(0)
        train_corrects += torch.sum(pre_lab == group.to(DEVICE).data)
    #计算一个epoch的损失和精度
    train_loss = train_loss_epoch/len(train_data.groups)
    train_acc = train_corrects.double()/len(train_data.groups)
    print("---------------------------------------------------")
    print("epoch:",epoch,"train_loss:",train_loss,"train_acc:",train_acc)
     #计算在验证集上的表现
    MyResnet.eval()
    for step,(mri,group) in enumerate(test_loader):
        output = MyResnet(mri)
        loss = loss_func(output,group.long())
        pre_lab = torch.argmax(output,1).to(DEVICE)
        val_loss_epoch += loss.item()*group.size(0)
        val_corrects += torch.sum(pre_lab == group.to(DEVICE).data)

    #计算一个epoch上的输出loss和acc
    val_loss = val_loss_epoch/len(test_data.groups)
    val_acc = val_corrects.double()/len(test_data.groups)
    print("epoch:",epoch,"val_loss:",val_loss,"val_acc:",val_acc)
    #保存每个epoch上的输出loss和acc
    historyl.log(epoch,train_loss=train_loss,val_loss = val_loss,train_acc = train_acc.item(),val_acc = val_acc.item())
    #可视化网络训练的过程
    # with canvasl:
    #     canvasl.draw_plot([historyl["train_loss"],historyl["val_loss"]])
    #     canvasl.draw_plot([historyl["train_acc"],historyl["val_acc"]])


---------------------------------------------------
epoch: 0 train_loss: 1.0799084191340276 train_acc: tensor(0.4454, device='cuda:0', dtype=torch.float64)
epoch: 0 val_loss: 1.0120258224010468 val_acc: tensor(0.5250, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 1 train_loss: 0.9806409188764756 train_acc: tensor(0.5533, device='cuda:0', dtype=torch.float64)
epoch: 1 val_loss: 1.1306686079502106 val_acc: tensor(0.2850, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 2 train_loss: 0.8434424131099074 train_acc: tensor(0.6248, device='cuda:0', dtype=torch.float64)
epoch: 2 val_loss: 1.178179018497467 val_acc: tensor(0.3000, device='cuda:0', dtype=torch.float64)
---------------------------------------------------
epoch: 3 train_loss: 0.7042753584262269 train_acc: tensor(0.6989, device='cuda:0', dtype=torch.float64)
epoch: 3 val_loss: 1.0264334082603455 val_acc: tensor(0.4950, device='cuda:0'